## 문제: 트윗 정서 분석

트윗의 정서를 분석해보자.
트윗은 API를 사용하여 수집할 수 있으나, 프로그램도 작성해야 하고 수집에 상당한 시간이 필요하다.
또한 트윗 메시지에 대해 긍정인지, 부정인지 정서를 하나 하나 판단해서 태깅해야 한다.
스탠포드 대학원생들이 수집해 놓은 트윗이 있다. 오픈소스가 아니므로 주의해야 한다 http://help.sentiment140.com/for-students/
이 데이터의 항목은:
* 트윗 정서 (0=부정, 2=중립, 4=긍정)
* 트윗 ID
* 트윗 일자
* 조회 (없으면 NO_QUERY)
* 사용자
* 트윗 텍스트

1) 전체 데이터 갯수, 각 '부정' '긍정' '중립' 별 데이터 갯수  
2) 트윗 정서 컬럼을 'label'로 변경  
5) stopwords 제거하고, 출력  
6) pipeline으로 tf-idf 계산하고 'features' 컬럼으로 변경  

https://github.com/tthustla/twitter_sentiment_analysis_part2/blob/master/Capstone_part3-Copy1.ipynb
https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-2-333514854913

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
import os

### DataFrame 생성

현재 작업디렉토리 아래 data/stanford 폴더 아래 csv 파일을 읽어 DataFrame을 생성하자.

In [5]:
tweetDf = spark.read\
    .format('com.databricks.spark.csv')\
    .options(header='false', inferschema='true')\
    .load(os.path.join("data", "트윗.csv"))

스키마가 자동인식되었는지 확인하자.

In [6]:
tweetDf.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



트윗의 건수를 세어보자.

In [7]:
tweetDf.count()

498

### 컬럼명 수정

컬럼명을 label, ID, tweetTime, query, user, text로 변경한다.

In [9]:
tweetDf = tweetDf.withColumnRenamed("_c0", "label") # 0=neg, 2=0, 4=pos
tweetDf = tweetDf.withColumnRenamed("_c1", "ID")
tweetDf = tweetDf.withColumnRenamed("_c2", "tweetTime")
tweetDf = tweetDf.withColumnRenamed("_c3", "query")
tweetDf = tweetDf.withColumnRenamed("_c4", "user")
tweetDf = tweetDf.withColumnRenamed("_c5", "text")

In [10]:
tweetDf.show(5)

+-----+---+--------------------+-------+--------+--------------------+
|label| ID|           tweetTime|  query|    user|                text|
+-----+---+--------------------+-------+--------+--------------------+
|    4|  3|Mon May 11 03:17:...|kindle2|  tpryan|@stellargirl I lo...|
|    4|  4|Mon May 11 03:18:...|kindle2|  vcu451|Reading my kindle...|
|    4|  5|Mon May 11 03:18:...|kindle2|  chadfu|Ok, first assesme...|
|    4|  6|Mon May 11 03:19:...|kindle2|   SIX15|@kenburbary You'l...|
|    4|  7|Mon May 11 03:21:...|kindle2|yamarama|@mikefish  Fair e...|
+-----+---+--------------------+-------+--------+--------------------+
only showing top 5 rows



### 텍스트 출력

트윗은 사람들이 편하게 작성한 텍스트라서, 줄임말, 구어, 유행어, 이모티콘 등이 뒤섞여 있을 수 밖에 없다.
몇 건만 출력을 해서 보더라도 그렇다는 것을 확인할 수 있다.
* "I've"는 "I have"의 줄임말이다. 불용어로 제거될 수 있다.
* "looooovvvveee"는 "love"의 강조하기 위해 쓰인 것으로 보인다.
* 단어 뒤에 "!!!"와 같은 강조가 발견된다.
* 이모티콘 ":)"도 볼 수 있다.

In [11]:
tweetDf.select('text').show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.                             |
|Reading my kindle2...  Love it... Lee childs is good read.                                                                                  |
|Ok, first assesment of the #kindle2 ...it fucking rocks!!!                                                                                  |
|@kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)|

### 결측값

분석에 앞서 결측이 있는지 보는 것이 중요하다. 데이터에 자체에 또는 수집 과정에서 결측이 발생할 수 있고, 분석과정에서 영향을 미칠 수 있기 때문이다.
* tweetDf.columns은 모든 컬럼을 조회하고,
* 이들 컬럼에 대해 반복문으로 수행하면서,
* isnan(컬럼) 또는 col(c).isNull()인지 개수를 세어서 count(),
* 컬럼명으로 출력하기 위해 alias() 하고,
* select() 명령어에 리스트로 넘겨서 출력한다.

수행하는 명령문이 복잡하면서도 짧게 작성이 되어 있다. Python의 특징을 잘 보여주고 있다.
그 결과 트윗에 결측은 없다.

In [12]:
from pyspark.sql.functions import isnan, when, count, col

tweetDf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in tweetDf.columns]).show()

+-----+---+---------+-----+----+----+
|label| ID|tweetTime|query|user|text|
+-----+---+---------+-----+----+----+
|    0|  0|        0|    0|   0|   0|
+-----+---+---------+-----+----+----+



또는 Dictionary 타입으로 만들어서 (키, null 개수)를 저장할 수 있다.

In [13]:
{c: tweetDf.where(tweetDf[c].isNull()).count() for c in tweetDf.columns}

{'label': 0, 'ID': 0, 'tweetTime': 0, 'query': 0, 'user': 0, 'text': 0}

### 일자별 트윗건수

#### 트윗일자 형변환

트윗일자는 ```Mon May 11 03:17:40 UTC 2009``` 형식으로 표현된다.

In [14]:
tweetDf.select('tweetTime').show(5, truncate=False)

+----------------------------+
|tweetTime                   |
+----------------------------+
|Mon May 11 03:17:40 UTC 2009|
|Mon May 11 03:18:03 UTC 2009|
|Mon May 11 03:18:54 UTC 2009|
|Mon May 11 03:19:04 UTC 2009|
|Mon May 11 03:21:41 UTC 2009|
+----------------------------+
only showing top 5 rows



#### Python 요일, 일시 형식으로 변환

Python에서 요일, 일시는 다양한 표현형식으로 출력할 수 있다.
아래 표를 참조하자.

표시 | 설명               |예
---|--------------------|--------------------
%a | 요일 줄임말           | Sun, Mon, ..., Sat
%b | 월 줄임말            | Jan, Feb, ..., Dec
%d | 숫자로 표현한 2글자 일자 | 01, 02, ..., 31
%H | 숫자로 표현한 2자리 시 | 00, 01, ..., 23
%M | 숫자로 표현한 2자리 분 | 00, 01, ..., 59
%S | 숫자로 표현한 2자리 초 | 00, 01, ..., 59
%d | 숫자로 표현한 2자리 일자 | 01, 02, ..., 31
%m | 숫자로 표현한 2자리 월  | 01, 02, ..., 12
%y | 숫자로 표현한 2자리 년수 (세기 불포함) | 00, 01, ..., 99
%Y | 숫자로 표현한 4자리 년수 (세기 포함)  | 0001, 0001, ..., 2020

연습으로 트윗일자를 일정한 표현형식으로 인식하여, 다른 형식 "년-월-일-시-분-초"로 변경하여 출력해 보자.

In [15]:
from datetime import datetime

_tweetTime = 'Mon May 11 03:17:40 UTC 2009'
datetime.strftime(datetime.strptime(_tweetTime,'%a %b %d %H:%M:%S UTC %Y'), '%Y-%m-%d %H:%M:%S')

'2009-05-11 03:17:40'

#### Spark에서 요일, 일시 형식으로 변환

트윗일자는 -> timestamp로 변환하고 -> 그리고 DateType()으로 변환한다.

먼저 트윗일자를 timestamp로 변환해보자.
트윗일자를 패턴으로 인식하자.
* EEE: 3글자 요일 예: Mon
* MMM: 3글자 월 예: Jan
* dd: 2글자 일자 예: 01, 02, ..., 31
* z: timezone 예: GMT+02:00; EET

```SparkUpgradeException... due to the upgrading of Spark 3.0```라는 오류가 발생하면
**```spark.sql.legacy.timeParserPolicy=LEGACY```**라고 설정을 변경해주어야 한다.

In [16]:
from pyspark.sql import functions as F

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
_tweetDf = tweetDf.withColumn('timestamp', F.unix_timestamp(tweetDf.tweetTime, "EEE MMM dd HH:mm:ss Z yyyy").alias())

In [17]:
_tweetDf.show(3)

+-----+---+--------------------+-------+------+--------------------+----------+
|label| ID|           tweetTime|  query|  user|                text| timestamp|
+-----+---+--------------------+-------+------+--------------------+----------+
|    4|  3|Mon May 11 03:17:...|kindle2|tpryan|@stellargirl I lo...|1242011860|
|    4|  4|Mon May 11 03:18:...|kindle2|vcu451|Reading my kindle...|1242011883|
|    4|  5|Mon May 11 03:18:...|kindle2|chadfu|Ok, first assesme...|1242011934|
+-----+---+--------------------+-------+------+--------------------+----------+
only showing top 3 rows



timestamp를 Spark의 DateType()으로 변환한다.

In [18]:
from pyspark.sql.types import DateType
_tweetDf = _tweetDf.withColumn('date', F.from_unixtime('timestamp').cast(DateType()))

In [19]:
_tweetDf.show(3)

+-----+---+--------------------+-------+------+--------------------+----------+----------+
|label| ID|           tweetTime|  query|  user|                text| timestamp|      date|
+-----+---+--------------------+-------+------+--------------------+----------+----------+
|    4|  3|Mon May 11 03:17:...|kindle2|tpryan|@stellargirl I lo...|1242011860|2009-05-11|
|    4|  4|Mon May 11 03:18:...|kindle2|vcu451|Reading my kindle...|1242011883|2009-05-11|
|    4|  5|Mon May 11 03:18:...|kindle2|chadfu|Ok, first assesme...|1242011934|2009-05-11|
+-----+---+--------------------+-------+------+--------------------+----------+----------+
only showing top 3 rows



#### 년, 월, 일 컬럼

방금 생성한 date 컬럼에서 년, 월, 일을 분리하여 컬럼으로 만들어 보자.

In [20]:
_tweetDf = _tweetDf\
    .withColumn('year', F.year("date").alias())\
    .withColumn('month', F.month("date").alias())\
    .withColumn('day', F.dayofmonth("date").alias())\

In [21]:
_tweetDf.show(5)

+-----+---+--------------------+-------+--------+--------------------+----------+----------+----+-----+---+
|label| ID|           tweetTime|  query|    user|                text| timestamp|      date|year|month|day|
+-----+---+--------------------+-------+--------+--------------------+----------+----------+----+-----+---+
|    4|  3|Mon May 11 03:17:...|kindle2|  tpryan|@stellargirl I lo...|1242011860|2009-05-11|2009|    5| 11|
|    4|  4|Mon May 11 03:18:...|kindle2|  vcu451|Reading my kindle...|1242011883|2009-05-11|2009|    5| 11|
|    4|  5|Mon May 11 03:18:...|kindle2|  chadfu|Ok, first assesme...|1242011934|2009-05-11|2009|    5| 11|
|    4|  6|Mon May 11 03:19:...|kindle2|   SIX15|@kenburbary You'l...|1242011944|2009-05-11|2009|    5| 11|
|    4|  7|Mon May 11 03:21:...|kindle2|yamarama|@mikefish  Fair e...|1242012101|2009-05-11|2009|    5| 11|
+-----+---+--------------------+-------+--------+--------------------+----------+----------+----+-----+---+
only showing top 5 rows



#### 년별, 월별 트윗건수

트윗에 작성된 글은 시대상을 반영하기 때문에 작성일시를 확인해보자.
pivot을 해서, 트윗의 작성년월을 조회해 볼 수 있다. 트윗은 2009년 5, 6월에 작성되었고 건수도 분배되어 수집한 것을 알 수 있다.

In [22]:
_tweetDf.groupBy('year').pivot('month').count().show()

+----+---+---+
|year|  5|  6|
+----+---+---+
|2009|248|250|
+----+---+---+



### 긍정, 부정

#### 긍정, 부정별 건수

트윗은 정서가 판단되어, label이 붙어있다. 그 개수를 조회해 보자.

In [23]:
_tweetDf.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    4|  182|
|    2|  139|
|    0|  177|
+-----+-----+



#### 중립 filter

부정 0과 긍정 4의 경우만 선별하여 분석을 해보자.

In [24]:
_tweetDf = _tweetDf.filter(_tweetDf.label != 2)

중립 2는 제거되어, 출력에 나타나지 않고 있다.

In [25]:
_tweetDf.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    4|  182|
|    0|  177|
+-----+-----+



#### 단어분리

"text"를 단어로 분리해서 "words" 컬럼으로 만들자.

In [26]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
tokDf = tokenizer.transform(_tweetDf)

#### 이음동의 처리

앞 장에서 작성했던 함수를 호출하여, 아래 이음동의와 불필요한 단어를 처리해보자.
* loooooooovvvvvveee -> love, 
* :) -> smile

단, Pipeline에 udf를 단계로 추가하려면 ```SQLTransformer```를 사용해야 된다.
명령어가 복잡해지므로 앞서 tokenizer는 pipeline에 넣지 않고 transform()을 실행하고, tokDf에 대해 udf를 실행한다.

In [27]:
import re

def trim(wordList):
    regex = re.compile('\d+')
    cleaned=list()
    for w in wordList:
        if not regex.match(w):
            cleaned.append(w.lstrip('‘').rstrip("’").rstrip(',').rstrip('.')\
                           .replace("loooooooovvvvvveee","love")\
                           .replace(":)","smile"))
    return cleaned

In [28]:
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType, StringType

trimUdf=f.udf(trim, ArrayType(StringType()))

In [29]:
_tweetDf = tokDf.withColumn('words', trimUdf(f.col('tokens')))

#### 불용어 제거

트윗에는 줄임말, 구어, 이모티콘 등을 포함하고 있어, 공식적으로 작성된 문서와는 다르다.

In [30]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="words", outputCol="nostops")

In [31]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

영어 "I've"를 제거한다. 영어 불용어 사전에 포함되이 있지만, 사용방법을 배우기 위해 넣었다.
실제 트윗에서 불용어에 해당되는 단어를 스스로 찾아서 추가할 수 있다.

In [32]:
_mystopwords=["I've"]
for e in _mystopwords:
    stopwords.append(e)

In [33]:
stop.setStopWords(stopwords)
#stopDf=stop.transform(tokDf)
#stopDf.show()

StopWordsRemover_a3aba6a2c852

#### TF-IDF

In [34]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer

hashtf = HashingTF(numFeatures=2**16, inputCol="nostops", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

#### Pipeline

위 tokenizer, stop, hashtf, idf를 단계적으로 처리한다.

In [35]:
from pyspark.ml import Pipeline

#label_stringIdx = StringIndexer(inputCol = "_label", outputCol = "label")
#pipeline = Pipeline(stages=[tokenizer, stop, hashtf, idf])
pipeline = Pipeline(stages=[stop, hashtf, idf])

In [36]:
pipelineFit = pipeline.fit(_tweetDf)

_tweetDfDone = pipelineFit.transform(_tweetDf)
#valDf = pipelineFit.transform(val)

In [37]:
_tweetDfDone.show(5)

+-----+---+--------------------+-------+--------+--------------------+----------+----------+----+-----+---+--------------------+--------------------+--------------------+--------------------+--------------------+
|label| ID|           tweetTime|  query|    user|                text| timestamp|      date|year|month|day|              tokens|               words|             nostops|                  tf|            features|
+-----+---+--------------------+-------+--------+--------------------+----------+----------+----+-----+---+--------------------+--------------------+--------------------+--------------------+--------------------+
|    4|  3|Mon May 11 03:17:...|kindle2|  tpryan|@stellargirl I lo...|1242011860|2009-05-11|2009|    5| 11|[@stellargirl, i,...|[@stellargirl, i,...|[@stellargirl, lo...|(65536,[2568,2701...|(65536,[2568,2701...|
|    4|  4|Mon May 11 03:18:...|kindle2|  vcu451|Reading my kindle...|1242011883|2009-05-11|2009|    5| 11|[reading, my, kin...|[reading, my, kin...

아래에서 보듯이 love, smile이 출력되고 있다.

In [38]:
_tweetDfDone.select("nostops").show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                     |
+------------------------------------------------------------------------------------------------------------+
|[@stellargirl, love, kindle2, dx, cool, fantastic, right]                                                   |
|[reading, kindle2, , love, lee, childs, good, read]                                                         |
|[ok, first, assesment, #kindle2, ...it, fucking, rocks!!!]                                                  |
|[@kenburbary, love, kindle2, mine, months, never, looked, back, new, big, one, huge!, need, remorse!, smile]|
|[@mikefish, , fair, enough, kindle2, think, perfect, , smile]                                               |
+------------------------------------------------------------------------------------------------------------+
o

훈련과 테스트로 구분한다. 그 비율은 훈련 70%, 테스트 30%이다.

```python
(trainingData, testData) = data.randomSplit([0.7, 0.3])
```

In [39]:
(trainDf, valDf, testDf) = _tweetDfDone.randomSplit([0.6, 0.2, 0.2], seed = 2000)

### NB 분류

Spark에서는 pyspark.ml.classification의 NaiveBayes 모델을 사용한다.
modelType은 "multinomial"로 설정한다.

In [40]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='features', labelCol='label', modelType='multinomial', predictionCol='prediction')

In [41]:
model = nb.fit(trainDf)

In [42]:
predictions=model.transform(trainDf)

In [43]:
predictions.select('label','text','prediction').show()

+-----+--------------------+----------+
|label|                text|prediction|
+-----+--------------------+----------+
|    0|Fuck this economy...|       0.0|
|    0|@Karoli I firmly ...|       0.0|
|    0|dear nike, stop w...|       0.0|
|    0|Played with an an...|       0.0|
|    0|US planning to re...|       0.0|
|    0|omg so bored &amp...|       1.0|
|    0|I'm itchy and mis...|       1.0|
|    0|@sekseemess no. I...|       1.0|
|    0|Blah, blah, blah ...|       0.0|
|    0|started to think ...|       1.0|
|    0|annoying new tren...|       0.0|
|    0|omg. The commerci...|       0.0|
|    0|@morind45 Because...|       0.0|
|    0|yahoo answers can...|       1.0|
|    0|RT @SmartChickPDX...|       1.0|
|    0|needs someone to ...|       0.0|
|    0|@legalgeekery Yea...|       1.0|
|    0|Damn you North Ko...|       0.0|
|    0|Why the hell is P...|       1.0|
|    0|"Are YOU burning ...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



In [44]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [45]:
evaluator.evaluate(predictions)

0.8280470553197826

### Logistic Regression

In [46]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=100)

In [47]:
lrModel = lr.fit(trainDf)

In [48]:
predictions = lrModel.transform(valDf)

In [49]:
predictions.select('label','prediction').show(1000)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       4.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       4.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       4.0|
|    0|       4.0|
|    0|       4.0|
|    0|       0.0|
|    0|       4.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       0.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|       0.0|
|    4|       4.0|
|    4|       4.0|
|    4|       4.0|
|    4|     

### 평가

In [50]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.3701298701298702

In [52]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(valDf.count())

In [53]:
accuracy

0.819672131147541